In [55]:
!export WEB3_INFURA_PROJECT_ID=ad89ee0df87340bba88c0608c5cf2fc2

In [64]:
import pandas as pd
import json
import requests
from web3 import Web3
w3 = Web3(Web3.WebsocketProvider("wss://mainnet.infura.io/ws/v3/ad89ee0df87340bba88c0608c5cf2fc2"))

In [65]:
w3.eth.getBalance(w3.toChecksumAddress('0x5abfec25f74cd88437631a7731906932776356f9'))

9746121377372246792

# Looking at ETH Distribution

sources: 
- \[1\] [Ethereum blog: Launching the Ether Sale by Vitalik](https://blog.ethereum.org/2014/07/22/launching-the-ether-sale/)
- \[2\] [Ethereum AMA](https://www.reddit.com/r/IAmA/comments/2bjmgb/hi_we_are_the_ethereum_project_team_ask_us/)
- \[3\] [AVSA Tweet about contributors](https://twitter.com/avsa/status/1210587580741439489)

APIs:
- [Infura](https://infura.io)
- [ENS subgraph](https://thegraph.com/explorer/subgraph/ensdomains/ens?query=Example%20query)
    

### ICO Details \[1\]:
- tranches:
    - 0-14 days: 2000 ETH per BTC
    - linear decline: 1337 ETH per BTC
- lasts 42 days
- annual issuance is 0.26x initial ETH sold
- two endowment pools (each 0.099x initial ETH sold):
    - early contributors to the project
    - long-term endowment to non-profit foundation
- reserved right to use up to 5000 BTC from pre-sale while pre-sale is running to ramp development
- exodus 3-4 multisig address: `36PrZ1KHYMpqSyAQXSG8VwbUiq2EogxLo`


## Pre-mine Analysis

In [21]:
# Bring in Mainnet Genesis file
with open('./eth_distribution/frontier.json', 'r') as f:
    genesis_json = json.load(f)
genesis_txns = genesis_json['alloc']
genesis_txns = [{"address": key, "balance": value['balance']} for key, value in genesis_txns.items()]
genesis_txns = pd.DataFrame(genesis_txns)
genesis_txns['balance'] = genesis_txns['balance'].apply(float) / 1e18
genesis_txns['address'] = '0x' + genesis_txns['address']
genesis_txns.sort_values(by='balance', ascending=False)

,address,balance
6698,0x5abfec25f74cd88437631a7731906932776356f9,1.190148e+07
2197,0x1937c5c515057553ccbd46d5866455ce66290284,1.000000e+06
7364,0x11172b278ddd44eea2fdf4cb1d16962391c453d9,9.359000e+05
5414,0xff7843c7010aa7e61519b762dfe49124a76b0e4e,9.335800e+05
164,0x5281733473e00d87f11e9955e589b59f4ac28e7a,6.603600e+05
...,...,...
8166,0xf2c2904e9fa664a11ee25656d8fd2cc0d9a522a0,1.337000e+01
1399,0x7641f7d26a86cddb2be13081810e01c9c83c4b20,1.337000e+01
3177,0x33581cee233088c0860d944e0cf1ceabb8261c2e,1.337000e+01
4259,0x5ed3f1ebe2ae6756b5d8dc19cad02c419aa5778b,0.000000e+00


In [52]:
headers = {}
# The GraphQL query (with a few aditional bits included) itself defined as a multi-line string.       
query = """
{
  domains(where: {
    owner: "%s"
  }){
    id
    name
    labelName
    labelhash
  }
}
"""

def run_query(query, address): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.thegraph.com/subgraphs/name/ensdomains/ens', json={'query': query % (address)}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

def pull_ens_name(result):
    if len(result['data']['domains']) != 0:
        return result['data']['domains'][0]['name']
    else:
        return ''

def query_and_pull(address):
    return pull_ens_name(run_query(query, address))

result = query_and_pull('0xb913626032140a86c77d1fdde4f611a00d589c55') # Execute the query
# result = query_and_pull('0x5abfec25f74cd88437631a7731906932776356f9')
print(result)

statusweb.eth


In [48]:
first_200 = genesis_txns.iloc[:200,:].sort_values(by="balance", ascending=False).copy()
first_200['ens'] = first_200.address.apply(query_and_pull)
first_200

,address,balance,ens
164,0x5281733473e00d87f11e9955e589b59f4ac28e7a,660360.0,
171,0x3a72d635aadeee4382349db98a1813a4cfeb3df1,200000.0,
195,0xb8cc0f060aad92d4eb8b36b3b95ce9e90eb383d7,150000.0,
183,0x5816c2687777b6d7d2a2432d59a41fa059e3a406,133700.0,
112,0xfda3042819af3e662900e1b92b4358eda6e92590,118200.0,
...,...,...,...
140,0x550aadae1221b07afea39fba2ed62e05e5b7b5f9,20.0,
121,0x9eb281c32719c40fdb3e216db0f37fbc73a026b7,20.0,
155,0x873e49135c3391991060290aa7f6ccb8f85a78db,20.0,
110,0x9af5c9894c33e42c2c518e3ac670ea9505d1b53e,18.2,


In [50]:
first_200[first_200.ens != '']

,address,balance,ens


In [66]:
def pull_current_balance(address):
    return float(w3.eth.getBalance(w3.toChecksumAddress(address))) / 1e18


In [68]:
pull_current_balance('0x3a72d635aadeee4382349db98a1813a4cfeb3df1')

0.99895

In [71]:
genesis_txns['current_balance'] = genesis_txns['address'].apply(pull_current_balance)

In [73]:
genesis_txns['perc_remains'] = genesis_txns['current_balance'] / genesis_txns['balance'] * 100

In [74]:
genesis_txns.head()

,address,balance,current_balance,perc_remains
0,0x3282791d6fd713f1e94f4bfd565eaa78b3a0599d,1337.000,1337.000000,100.000000
1,0x17961d633bcf20a7b029a7d94b7df4da2ec5427f,229.427,0.000000,0.000000
2,0x493a67fe23decc63b10dda75f3287695a81bd5ab,880.000,16.719346,1.899926
3,0x01fb8ec12425a04f813e46c54c05748ca6b29aa9,259.800,0.005233,0.002014
4,0xd2a030ac8952325f9e1db378a71485a24e1b07b2,2000.000,0.000000,0.000000


In [79]:
genesis_txns.to_pickle('genesis_txns.p')
genesis_txns.to_csv('genesis_txns.csv')

In [76]:
genesis_txns['ens'] = genesis_txns.address.apply(query_and_pull)

In [82]:
genesis_txns.sort_values(by="current_balance", ascending=False)

,address,balance,current_balance,perc_remains,ens
2763,0x1b3cb81e51011b549d78bf720b0d924ac763a7c2,560000.0,560000.000030,100.000000,
5730,0x2b6ed29a95753c3ad948348e3e7b1a251080ffb9,250000.0,250000.007046,100.000003,
6669,0x51f9c432a4e59ac86282d6adab4c2eb8919160eb,530000.0,218899.998330,41.301886,
872,0x7f3a1e45f67e92c880e573b43379d71ee089db54,100000.0,100000.000010,100.000000,
1071,0x8b505e2871f7deb7a63895208e8227dcaa1bff05,61216.6,61216.600000,100.000000,
...,...,...,...,...,...
531,0x2efc4c647dac6acac35577ad221758fef6616faa,8000.0,0.000000,0.000000,
3920,0x7517f16c28d132bb40e3ba36c6aef131c462da17,18.2,0.000000,0.000000,
6542,0x21546914dfd3af2add41b0ff3e83ffda7414e1e0,5969.1,0.000000,0.000000,
6544,0x6fd98e563d12ce0fd60f4f1f850ae396a9823c02,1261.0,0.000000,0.000000,


### Open contributions

### Contributor Allocations

source: \[3\]
- Vitalik: Ethereum 
- Hoskinson: Classic
- Di'orio: Jaxx
- Mihai: Akasha
- Gavin: Polkadot
- Lubin: Consensys
- Jeff: Not involved anymore
- Amir: seems to be an angel investor now?
- Koblitz: cryptography professor (same as before)
- Ralph Merkle: same as above
- Stephan Tual: Slockit, Atlas Neue
- Taylor: volunteers for swarm team
- Jeremy Wood: Cardano
- Mathias: Braveno (cryptoexchange)
- Ryan: Dash
- Fierro: Mimir (lab building blockchain apps?)
- Stott: couldn’t find
- Meikle: Ethereum
- Colby: vDice
- Sureanu: Akasha
- Hidskes: couldnt find
- Mohan: Themys (blockchain risk assessment?)
- Houwu Chen: seems to be a researcher still?
- Nguyen: couldn’t find. There’s a "Forbes under 30” with his name, same guy?
- Wilding: L4, a web3 studio that built ETHGlobal
- Gluck: Riot games
- Chow: still at ethereum I believe?

### Summary